# Introduction: Model and protocols

Welcome!

In this tutorial, we'll look at fitting models of ionic currents using voltage-clamp data, i.e., currents measured in experiments where a cell's membrane potential is controlled by the experimenter.

The examples start quite simple, but build up in complexity as we go along.

## The model

The next section describes the model equations. If you like, you can skip past this and go straight to the model implementation below.

For all of this tutorial, we use a simple two-gate Hogdkin-Huxley model to describe channel kinetics, and an Ohmic term to describe the driving force:

$$I(V, t) = g_\text{max} \cdot a(V, t) \cdot r(V, t) \cdot (V - E)$$

Here $g_\text{max}$ is the maximum conductance, determined by the number of channels and the conductance per channel, but assumed constant for this tutorial.

The variables $a$ and $r$ are dimensionless variables that vary between 0 and 1, and describe the fraction of channels in an _activated_ (a) or _recovered_ (r) state.
The underlying assumpution is that the current kinetics are described by two independent processes called "activation" and "inactivation", and that channels must be "activated" and "recovered from inactivation" to conduct.

Taken together, $g_\text{max} \cdot a \cdot r$ represent the combined conductance of the channel, so that $I = g(V, t) \cdot (V - E)$, or $I = g \Delta{V}$.
In other words, we assume that when a channel is open, the current through the channel is a passive current, linearly proportional to a voltage difference $(V - E)$, where $E$ is the Nernst or reversal potential, and is determined by the balance of _diffusion_ (particles moving randomly will eventually spread out more or less evenly) and the electrotonic attraction and repulsion caused by the particles' charges.

For most of this tutorial, we'll assume that $g_\text{max}$ and the parameters in the equations for $a$ and $r$ need to be identifier, while the reversal potential $E$ can be estimated from the internal and external solutions used in the experiment.

<img src="resources/gating.png" width="552px" />

The variables $a$ and $b$ are both described as simple chemical processes that can be in one of two states: _open_ (O) or _closed_ (C).
Each state has a certain energy associated with it, and to make the transition between the two states an energy barrier needs to be overcome.
The size of this barrier determines the transition rates in both directions $k_1$ and $k_2$.

In (physical) chemistry, this is commonly described using the transition state theory, e.g. with the [Eyring equation](https://en.wikipedia.org/wiki/Eyring_equation) (with an added term for the electrical energy, see [Tsien & Noble 1969](https://link.springer.com/article/10.1007/BF01869785)).
The resulting rate equations contain a mixture of physical constants and model parameters, but can be reduced to the form

$$k_i = a_i \cdot e ^ {b_i \cdot V} $$


We can use rate equations of this form to write a differential equation for activation.
Using $a$ for the fraction of activated channels, we see that the fraction of channels _not_ activation must be $1 - a$.
Then, setting $k_1$ as the rate at which channels activate, and $k_2$ as the rate at which channels deactivate, we can write

$$\frac{da}{dt} = (1 - a) k_1 - a k_2$$

Here, $a$ is dimensionless, while the rates $k_i$ have units of 1-over-time.
To get a bit more insight into the equation, it's commonly rewritten as

$$\frac{da}{dt} = k_1 - (k_1 + k_2) a = \frac{\frac{k1}{k1 + k2} - a}{\frac{1}{k_1 + k_2}} = \frac{a_\infty - a}{\tau_a}$$

This equation is positive when $a < a_\infty$, which will cause $a$ to move towards $a_\infty$ over time.
Similarly, it's negative when $a > a_\infty$, so that again $a$ moves towards $a_\infty$.
When $a = a_\infty$ the rate of change is zero.
In other words, $a_\infty$ is the action variable's _steady state_.

The rate at which it moves towards its steady state is determined by $\tau_a$, where a _larger_ value of $\tau_a$ will cause the rate to be _slower_.

Following [the 2017 paper by Beattie et al.](https://doi.org/10.1113/JP276068) we write a similar equation for the recovery variable as:

$$\frac{dr}{dt} = (1 - r) k_4 - r k_3 = \frac{r_\infty - r}{\tau_r}$$

### Note: structural unidentifiability

Instead of $a e ^ {b V}$ we could also write $e ^ {a' b V}$ (where $a' = \log a$).
If temperature is varied, you might encounter a form with three parameters (see e.g. [Lei et al., BiophysJ 2019](https://doi.org/10.1016/j.bpj.2019.07.030)).
But in some studies where only voltage is varied, you still might encounter forms like $a e ^ {b + cV}$.
Look out for these!
This can be rewritten as $e^{(\log a + b) + c V}$, where there are infinitely many combinations $\log a + b$ that give the same rate.
By using the equation in this form you introduce _structural unidentifiability_ into your model, which will make the later optimisation problems much much harder to do.


## The model implementation

Now that we've defined the maths, we code up the model using [myokit](http://myokit.org).

